In [1]:
import fsspec
import os
from ipfsspec.asyn import AsyncIPFSFileSystem
from fsspec import register_implementation
import asyncio
import io

# register_implementation(IPFSFileSystem.protocol, IPFSFileSystem)
register_implementation(AsyncIPFSFileSystem.protocol, AsyncIPFSFileSystem)

# with fsspec.open("ipfs://QmZ4tDuvesekSs4qM5ZBKpXiZGun7S2CYtEZRB3DYXkjGx", "r") as f:
#     print(f.read())
class fs:
    ipfs = fsspec.filesystem("ipfs")
    file = fsspec.filesystem("file")

Changed to public node


# Changing Gateways

In [2]:
fs.ipfs.change_gateway_type = 'public'

Changed to public node


In [3]:
fs.ipfs.gateway.gws

[(<ipfsspec.gateway.GatewayState at 0x7f503b620940>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7f503b620d30>),
 (<ipfsspec.gateway.GatewayState at 0x7f503b6204f0>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7f503b620910>),
 (<ipfsspec.gateway.GatewayState at 0x7f503b620370>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7f503b620f10>),
 (<ipfsspec.gateway.GatewayState at 0x7f503b620d00>,
  <ipfsspec.gateway.AsyncIPFSGateway at 0x7f503b620e80>)]

In [4]:
fs.ipfs.gateway.gws[0][1].url

'https://ipfs.io'

In [5]:
fs.ipfs.change_gateway_type = 'local'

Changed to local node


In [6]:
fs.ipfs.gateway.gws[0][1].url

'http://ipfs:8080'

# Local

## Put

#### Put File

In [7]:
# put in a file and store it into test folder 'fam'
put_file_res = fs.ipfs.put(path='test/data/input/yo.txt', rpath='/test_put_file')
fs.ipfs.expand_path('/test_put_file/yo.txt')

['/test_put_file/yo.txt']

#### Put Folder

In [8]:
# put a directory and store it into test folder
fs.ipfs.put(path='test/data/input', rpath='/test_put_folder' ,recursive=True, return_cid=False)
fs.ipfs.ls('/')

[{'name': '/test',
  'CID': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7',
  'type': 'directory',
  'size': 0},
 {'name': '/test_get_file',
  'CID': 'QmQFuaEFT1pmyDmh8bx5peApmXXoA5D3CxZ1HfxBztt1cj',
  'type': 'directory',
  'size': 0},
 {'name': '/test_get_folder',
  'CID': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7',
  'type': 'directory',
  'size': 0},
 {'name': '/test_put_file',
  'CID': 'QmQFuaEFT1pmyDmh8bx5peApmXXoA5D3CxZ1HfxBztt1cj',
  'type': 'directory',
  'size': 0},
 {'name': '/test_put_folder',
  'CID': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7',
  'type': 'directory',
  'size': 0},
 {'name': '/test_rm_file',
  'CID': 'QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn',
  'type': 'directory',
  'size': 0}]

## Cat

#### Cat File

In [9]:
# Given a file CID to cat
fs.ipfs.cat('QmNuMqLrDoSEZeZxyPWUjFZv9HVYwmHB9n5PEwpfoJyAD6')

b'yofvervevdfvd'

In [10]:
# Given a file mfs to cat
fs.ipfs.cat('/test_put_file/yo.txt')

b'yofvervevdfvd'

#### Cat Folder

In [11]:
# Given a directory CID to cat
fs.ipfs.cat('Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7')

{'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/bruh.txt': b'bruuhhhhhhhhh',
 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/yo.txt': b'yofvervevdfvd'}

In [12]:
# Given a file mfs to cat
fs.ipfs.cat('/test_put_folder')

{'/test_put_folder/bruh.txt': b'bruuhhhhhhhhh',
 '/test_put_folder/yo.txt': b'yofvervevdfvd'}

## Remove

#### Remove File

In [15]:
fs.ipfs.put(path='test/data/input/yo.txt', rpath='/test_rm_file' ,recursive=True, return_cid=False)
print('Before: ',fs.ipfs.ls('/test_rm_file', detail=False))
fs.ipfs.rm('/test_rm_file/yo.txt')
print('After: ',fs.ipfs.ls('/test_rm_file', detail=False))

Before:  ['/test_rm_file/yo.txt']
After:  []


#### Remove Folder

In [16]:
fs.ipfs.put(path='test/data', rpath='/test_rm_folder', recursive=True, return_cid=False)
print('Before: ',fs.ipfs.ls('/test_rm_folder', detail=False))
fs.ipfs.rm('/test_rm_folder')
print('After: ',fs.ipfs.ls('/test_rm_folder', detail=False))

Before:  ['/test_rm_folder/input', '/test_rm_folder/output', '/test_rm_folder/outputQmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7']
After:  []


## Get

#### Get Remote File - MFS Path

In [18]:
if fs.file.exists('test/data/yo.txt'):
    fs.file.rm('test/data/yo.txt')
    
fs.ipfs.put(lpath='test/data/input/yo.txt', rpath='/test_get_file')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('/test/data/*.txt')])
fs.ipfs.get(rpath='/test_get_file/yo.txt', lpath='test/data',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])

Before:  []
After:  ['test/data/yo.txt']


#### Get Remote File - CID Path 

*not working*

In [29]:
fs.ipfs.get(rpath='/test_get_file/yo.txt', lpath='test/data')

[None]

In [31]:
if fs.file.exists('test/data/yo.txt'):
    fs.file.rm('test/data/yo.txt')
    
cid = fs.ipfs.put(lpath='test/data/input/yo.txt', rpath='/test_get_file')
print(cid)
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])

fs.ipfs.get(rpath=cid', lpath='test/data',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/*.txt')])

QmNuMqLrDoSEZeZxyPWUjFZv9HVYwmHB9n5PEwpfoJyAD6
Before:  []
After:  []


#### Get Remote Folder - MFS Path

In [17]:
if fs.file.exists('test/data/output'):
    fs.file.rm('test/data/output', recursive=True)
    
fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])
fs.ipfs.get(rpath='/test_get_folder', lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

Before:  []
After:  ['test/data/output/bruh.txt', 'test/data/output/yo.txt']


#### Get Remote Folder - CID Path

In [18]:
if fs.file.exists('test/data/output'):
    fs.file.rm('test/data/output', recursive=True)
    
fs.ipfs.put(lpath='test/data/input', rpath='/test_get_folder')
print('Before: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

fs.ipfs.get(rpath='Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7', lpath='test/data/output',recursive=True, return_cid=False)
print('After: ', [p.lstrip(os.getcwd()) for p in fs.file.glob('test/data/output/*.txt')])

Before:  []
After:  ['test/data/output/bruh.txt', 'test/data/output/yo.txt']


# Public

In [19]:
fs.ipfs.change_gateway_type = 'public'

Changed to public node


## Cat

#### Cat File

In [20]:
# Given a file CID to cat
r = fs.ipfs.cat('QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9')

dict_keys(['gateway_type', 'url'])
QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9 {'name': 'QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9'} BRUH <ClientResponse(https://ipfs.io/ipfs/QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9) [200 OK]>
<CIMultiDictProxy('Server': 'openresty', 'Date': 'Wed, 13 Jul 2022 16:10:43 GMT', 'Content-Type': 'text/plain; charset=utf-8', 'Content-Length': '1139', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Access-Control-Allow-Methods': 'GET', 'Cache-Control': 'public, max-age=29030400, immutable', 'Etag': '"QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9"', 'X-Ipfs-Gateway-Host': 'ipfs-bank9-am6', 'X-Ipfs-Path': '/ipfs/QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9', 'X-Ipfs-Roots': 'QmP8jTG1m9GSDJLCbeWhVSVgEzCPPwXRdCRuJtQ5Tz9Kc9', 'X-IPFS-POP': 'ipfs-bank9-am6', 'X-Ipfs-DataSize': '1139', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, OPTIONS', 'Access-Control-Allow-Headers': 'X-Requested-With, Range, Content-Ran

#### Cat Folder

In [21]:
# Given a directory CID to cat
fs.ipfs.cat('Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7')

dict_keys(['gateway_type', 'url'])
Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7 {'name': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7'} BRUH <ClientResponse(https://ipfs.io/ipfs/Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7) [200 OK]>
<CIMultiDictProxy('Server': 'openresty', 'Date': 'Wed, 13 Jul 2022 16:10:47 GMT', 'Content-Type': 'text/html', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'Access-Control-Allow-Methods': 'GET', 'Etag': '"DirIndex-2b567f6r5vvdg_CID-Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7"', 'X-Ipfs-Gateway-Host': 'ipfs-bank9-am6', 'X-Ipfs-Path': '/ipfs/Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7', 'X-Ipfs-Roots': 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7', 'X-IPFS-POP': 'ipfs-bank9-am6', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': 'GET, POST, OPTIONS', 'Access-Control-Allow-Headers': 'X-Requested-With, Range, Content-Range, X-Chunked-Output, X-Stream-Output', 'Access-Control-Expose-Headers': 'Content-Range, X-Chunked

{'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/bruh.txt': b'bruuhhhhhhhhh',
 'Qmdgy8KEqoyU9fWdoqPTUDSYWiRsHZVxebkH7fKyJhQnW7/yo.txt': b'yofvervevdfvd'}

## Get

#### Get Remote File - CID Path 


#### Get Remote Folder - CID Path